In [ ]:
import pickle 
import warnings

# Filter all warnings
warnings.filterwarnings("ignore")

trained_regressors = {} 
table_data = []


# Loop through models and distributions and evaluate each one
for name, values in splitted_data.items():
    
    X_train_survival = values['X_train_survival']
    X_test_survival = values['X_test_survival']
    y_train_survival = values['y_train_survival']
    y_test_survival = values['y_test_survival']

    X_train = values['X_train']
    y_train = values['y_train']
    X_test = values['X_test']
    y_test = values['y_test']

    X_train_mn_ext_feat = values['X_train_mn_ext_feat']
    X_test_mn_ext_feat = values['X_test_mn_ext_feat']
    y_train_mn_ext_feat = values['y_train_mn_ext_feat'] 
    y_test_mn_ext_feat = values['y_test_mn_ext_feat']
    
    X_train_tsfel = values['X_train_tsfel']
    y_train_tsfel = values['y_train_tsfel']
    X_test_tsfel = values['X_test_tsfel']
    y_test_tsfel = values['y_test_tsfel']

    THRESHOLD = round(dataframes[name]['threshold'].iloc[0])

    results = pd.DataFrame(index=values['y_test'].index)

    if X_train_mn_ext_feat.isna().sum().sum() > 0:
        X_train_mn_ext_feat = X_train_mn_ext_feat.fillna(0)
    if X_test_mn_ext_feat.isna().sum().sum() > 0:
        X_test_mn_ext_feat = X_test_mn_ext_feat.fillna(0)

    for regression in list_of_models:
        for distribution in distributions:
            if regression not in [#'Prophet', 
                                #'Prophet_simple_threshold', 
                                'Survival_Analysis', 
                                #'hidden_markov_models', 
                                'Survival_Analysis_manual_extracted_features',
                                'Survival_Analysis_tsfel_extracted_features']:
                # Implement the model fitting and evaluation logic here
                # print('\n')
                # print('Distribution:', distribution.name)
                # print('Model:', str(regression))

                regression.fit(X_train, y_train)

                trained_regressors[str(regression)+'_'+str(distribution.name)]=[regression]

                # getting point forecasts
                point_forecasts = regression.predict(X_test)
                std = y_train.std()
                if distribution == lognorm:
                    exceedance_prob = np.asarray([1 - distribution.cdf(THRESHOLD, loc=x_, s=std) for x_ in point_forecasts])
                elif distribution == genextreme:
                    exceedance_prob = np.asarray([1 - distribution.cdf(THRESHOLD, c = -0.1, loc=x_, scale=std) for x_ in point_forecasts])
                elif distribution != lognorm:
                    exceedance_prob = np.asarray([1 - distribution.cdf(THRESHOLD, loc=x_, scale=std) for x_ in point_forecasts])
                
                #y_test_ = new_df['marked'].loc[y_test.index]
                y_test_ = (y_test > THRESHOLD).astype(int)
                
                y_pred_binary = np.where(exceedance_prob >= 0.5, 1, 0)
                results['actual_predictions_from_model'+'_'+str(regression)+'_'+str(distribution.name)]  = point_forecasts
                results['y_pred_binary'+'_'+str(regression)+'_'+str(distribution.name)]  = y_pred_binary
                trained_regressors[str(regression)+'_'+str(distribution.name)].append(y_pred_binary)

                # Compute the confusion matrix
                cm = confusion_matrix(y_test_, y_pred_binary)

                # Compute the precision, recall, and F1 score
                accuracy = accuracy_score(y_test_, y_pred_binary)
                precision = precision_score(y_test_, y_pred_binary)
                recall = recall_score(y_test_, y_pred_binary)
                f1 = f1_score(y_test_, y_pred_binary)
                roc_auc = roc_auc_score(y_test_, exceedance_prob)

                # Print the results
                # print(f"Confusion Matrix:\n{cm}")
                # print(f"Accuracy: {accuracy:.2f}")
                # print(f"Precision: {precision:.2f}")
                # print(f"Recall: {recall:.2f}")
                # print(f"F1 Score: {f1:.2f}")
                # print(f"Roc AUC: {roc_auc:.2f}")

                table_data.append([THRESHOLD, 
                                timesteps, 
                                distribution.name, 
                                regression, 
                                    str(int(np.round(accuracy, 2)*100))+str('%'), 
                                    str(int(np.round(precision, 2)*100))+str('%'), 
                                    str(int(np.round(recall, 2)*100))+str('%'), 
                                    str(int(np.round(f1, 2)*100))+str('%'), 
                                    str(int(np.round(roc_auc, 2)*100))+str('%'), 
                                    test_size])
                
            
            if regression == 'Survival_Analysis':
                    # Implement Survival Analysis logic here
                    trained_regressors[str(regression)]=[regression]

                    scores_cph_tree = {}
                    scores_RandomSurvivalForest = {} 

                    est_cph_tree = RandomSurvivalForest(random_state=random_state)
                    for i in range(1, 61):
                        n_estimators = i * 5
                        est_cph_tree.set_params(n_estimators=n_estimators)
                        est_cph_tree.fit(X_train_survival, y_train_survival)
                        scores_cph_tree[n_estimators] = est_cph_tree.score(X_test_survival, y_test_survival)
                        scores_RandomSurvivalForest['scores_cph_tree'+str(n_estimators)] =  pickle.dumps(est_cph_tree)
                    

                    metrics = []
                    prob_percentages = [0.85, 0.90]

                    for i in range(5, 61, 5):
                        for prob_percentage in prob_percentages:
                            list_of_lists = pickle.loads(scores_RandomSurvivalForest['scores_cph_tree'+str(i)]).predict_survival_function(X_test_survival, return_array=True)

                            y_test_ = (y_test_survival['extreme_event'] == True).astype(int)
                            y_pred_binary = pd.DataFrame(list_of_lists[0:,0], columns=['probabilities'])['probabilities'].apply(lambda x: 1 if x < prob_percentage else 0).values

                            # Calculate metrics
                            accuracy = accuracy_score(y_test_, y_pred_binary)
                            precision = precision_score(y_test_, y_pred_binary)
                            recall = recall_score(y_test_, y_pred_binary)
                            f1 = f1_score(y_test_, y_pred_binary)
                            roc_auc = roc_auc_score(y_test_, list_of_lists[0:,0])
                            

                            metrics.append({'model':'scores_cph_tree'+str(i),
                                            'accuracy': accuracy,
                                            'precision': precision,
                                            'recall': recall,
                                            'roc_auc':roc_auc,
                                            'f1-score':f1,
                                            'y_pred_binary':y_pred_binary})

                            print(accuracy,'%', precision,'%', recall,'%', f1,'%', roc_auc,'%')

                    # Sort the list based on the 'age' field
                    sorted_data = sorted(metrics, key=lambda x: x['f1-score'], reverse=True)

                    accuracy = sorted_data[0]['accuracy']
                    precision = sorted_data[0]['precision']
                    recall = sorted_data[0]['recall']    
                    f1 = sorted_data[0]['f1-score']
                    roc_auc = sorted_data[0]['roc_auc']

                    trained_regressors[str(regression)].append(y_pred_binary)
                    
                    table_data.append([ THRESHOLD, 
                                    timesteps, 
                                    None, 
                                    regression+'_'+str(sorted_data[0]['model']), 
                                    str(int(np.round(accuracy,2)*100))+str('%'), 
                                    str(int(np.round(precision,2)*100))+str('%'), 
                                    str(int(np.round(recall,2)*100))+str('%'), 
                                    str(int(np.round(f1,2)*100))+str('%'), 
                                    str(int(np.round(roc_auc,2)*100))+str('%'), 
                                    test_size])

                    
                    trained_regressors[str(regression)].append([accuracy, precision, recall, f1, roc_auc])

            if regression=='Survival_Analysis_manual_extracted_features':
                # Implement Survival Analysis logic here
                trained_regressors[str(regression)]=[regression]

                scores_cph_tree = {}
                scores_RandomSurvivalForest = {} 

                est_cph_tree = RandomSurvivalForest(random_state=random_state)
                for i in range(1, 61):
                    n_estimators = i * 5
                    est_cph_tree.set_params(n_estimators=n_estimators)
                    est_cph_tree.fit(X_train_mn_ext_feat, y_train_mn_ext_feat)
                    scores_cph_tree[n_estimators] = est_cph_tree.score(X_test_mn_ext_feat, y_test_mn_ext_feat)
                    scores_RandomSurvivalForest['scores_cph_tree'+str(n_estimators)] =  pickle.dumps(est_cph_tree)

                metrics = []
                prob_percentages = [0.85, 0.90]

                for i in range(5, 61, 5):
                    for prob_percentage in prob_percentages:
                        list_of_lists = pickle.loads(scores_RandomSurvivalForest['scores_cph_tree'+str(i)]).predict_survival_function(X_test_mn_ext_feat, return_array=True)

                        y_test_ = (y_test_survival['extreme_event'] == True).astype(int)
                        y_pred_binary = pd.DataFrame(list_of_lists[0:,0], columns=['probabilities'])['probabilities'].apply(lambda x: 1 if x < prob_percentage else 0).values

                        # Calculate metrics
                        accuracy = accuracy_score(y_test_, y_pred_binary)
                        precision = precision_score(y_test_, y_pred_binary)
                        recall = recall_score(y_test_, y_pred_binary)
                        f1 = f1_score(y_test_, y_pred_binary)
                        roc_auc = roc_auc_score(y_test_, list_of_lists[0:,0])
                        

                        metrics.append({'model':'scores_cph_tree'+str(i),
                                        'accuracy': accuracy,
                                        'precision': precision,
                                        'recall': recall,
                                        'roc_auc':roc_auc,
                                        'f1-score':f1,
                                        'y_pred_binary':y_pred_binary})

                        print(accuracy,'%', precision,'%', recall,'%', f1,'%', roc_auc,'%')

                # Sort the list based on the 'age' field
                sorted_data = sorted(metrics, key=lambda x: x['f1-score'], reverse=True)

                accuracy = sorted_data[0]['accuracy']
                precision = sorted_data[0]['precision']
                recall = sorted_data[0]['recall']    
                f1 = sorted_data[0]['f1-score']
                roc_auc = sorted_data[0]['roc_auc']

                trained_regressors[str(regression)].append(y_pred_binary)
                
                table_data.append([ THRESHOLD, 
                                    timesteps, 
                                    None, 
                                    regression+'_'+str(sorted_data[0]['model']), 
                                    str(int(np.round(accuracy,2)*100))+str('%'), 
                                    str(int(np.round(precision,2)*100))+str('%'), 
                                    str(int(np.round(recall,2)*100))+str('%'), 
                                    str(int(np.round(f1,2)*100))+str('%'), 
                                    str(int(np.round(roc_auc,2)*100))+str('%'), 
                                    test_size])

                
                trained_regressors[str(regression)].append([accuracy, precision, recall, f1, roc_auc])

            
            if regression=='Survival_Analysis_tsfel_extracted_features':
                # Implement Survival Analysis logic here
                trained_regressors[str(regression)]=[regression]

                scores_cph_tree = {}
                scores_RandomSurvivalForest = {} 

                est_cph_tree = RandomSurvivalForest(random_state=random_state)
                for i in range(1, 61):
                    n_estimators = i * 5
                    est_cph_tree.set_params(n_estimators=n_estimators)
                    est_cph_tree.fit(X_train_tsfel, y_train_tsfel)
                    scores_cph_tree[n_estimators] = est_cph_tree.score(X_test_tsfel, y_test_tsfel)
                    scores_RandomSurvivalForest['scores_cph_tree'+str(n_estimators)] =  pickle.dumps(est_cph_tree)

                metrics = []
                prob_percentages = [0.85, 0.90]

                for i in range(5, 61, 5):
                    for prob_percentage in prob_percentages:
                        list_of_lists = pickle.loads(scores_RandomSurvivalForest['scores_cph_tree'+str(i)]).predict_survival_function(X_test_tsfel, return_array=True)

                        y_test_ = (y_test_survival['extreme_event'] == True).astype(int)
                        y_pred_binary = pd.DataFrame(list_of_lists[0:,0], columns=['probabilities'])['probabilities'].apply(lambda x: 1 if x < prob_percentage else 0).values

                        # Calculate metrics
                        accuracy = accuracy_score(y_test_, y_pred_binary)
                        precision = precision_score(y_test_, y_pred_binary)
                        recall = recall_score(y_test_, y_pred_binary)
                        f1 = f1_score(y_test_, y_pred_binary)
                        roc_auc = roc_auc_score(y_test_, list_of_lists[0:,0])
                        

                        metrics.append({'model':'scores_cph_tree'+str(i),
                                        'accuracy': accuracy,
                                        'precision': precision,
                                        'recall': recall,
                                        'roc_auc':roc_auc,
                                        'f1-score':f1,
                                        'y_pred_binary':y_pred_binary})

                        print(accuracy,'%', precision,'%', recall,'%', f1,'%', roc_auc,'%')

                # Sort the list based on the 'age' field
                sorted_data = sorted(metrics, key=lambda x: x['f1-score'], reverse=True)

                accuracy = sorted_data[0]['accuracy']
                precision = sorted_data[0]['precision']
                recall = sorted_data[0]['recall']    
                f1 = sorted_data[0]['f1-score']
                roc_auc = sorted_data[0]['roc_auc']

                trained_regressors[str(regression)].append(y_pred_binary)
                
                table_data.append([ THRESHOLD, 
                                    timesteps, 
                                    None, 
                                    regression+'_'+str(sorted_data[0]['model']), 
                                    str(int(np.round(accuracy,2)*100))+str('%'), 
                                    str(int(np.round(precision,2)*100))+str('%'), 
                                    str(int(np.round(recall,2)*100))+str('%'), 
                                    str(int(np.round(f1,2)*100))+str('%'), 
                                    str(int(np.round(roc_auc,2)*100))+str('%'), 
                                    test_size])

                
                trained_regressors[str(regression)].append([accuracy, precision, recall, f1, roc_auc])


In [ ]:
from tabulate import tabulate

print(tabulate(table_data, headers=[
                                    "Threshold", 
                                    "Number of lags", 
                                    "Distribution", 
                                    "Algorithm", 
                                    "Accuracy", 
                                    "Precision", 
                                    "Recall", 
                                    "F1-score", 
                                    'Roc AUC', 
                                    'Test size'], tablefmt="grid"))

In [ ]:
import io 
from tabulate import tabulate

table_str = tabulate(table_data, headers=[
                                    "Threshold", 
                                    "Number of lags", 
                                    "Distribution", 
                                    "Algorithm", 
                                    "Accuracy", 
                                    "Precision", 
                                    "Recall", 
                                    "F1-score", 
                                    'Roc AUC', 
                                    'Test size'],tablefmt="pipe")

ingredient = 'fruits_and_verg'

def get_results_df():

    # Return the results dataframe
    df = pd.read_csv(io.StringIO(table_str), sep="|")
    df.to_csv(
              '/home/gmarinos/Documents/Code/extended_version_1st_pub/results/'+'_'+ingredient+'_'+'threshold'+'_'+str(np.round(THRESHOLD,0))+'.'+'csv', index=False)
    # results.to_csv('/home/gmarinos/Documents/Code/extended_version_1st_pub/results'+'_'+ingredient+'_'+'threshold'+'_'+str(np.round(THRESHOLD,0))+'.'+'csv')

In [ ]:
import pandas as pd 

all_results = pd.read_csv('/home/gmarinos/Documents/Code/extended_version_1st_pub/results/_fruits_and_verg_threshold_11.csv').drop(columns=['Unnamed: 0', 'Unnamed: 11']).iloc[1:]

In [ ]:
# Assuming df is your DataFrame with percentage values
all_results[' Accuracy   '] = all_results[' Accuracy   '].str.replace('%', '')
all_results[' Precision   '] = all_results[' Precision   '].str.replace('%', '')
all_results[' Recall   '] = all_results[' Recall   '].str.replace('%', '')
all_results[' F1-score   '] = all_results[' F1-score   '].str.replace('%', '')
all_results[' Roc AUC   '] = all_results[' Roc AUC   '].str.replace('%', '')

# Convert the columns to numeric values
all_results[' Accuracy   '] = pd.to_numeric(all_results[' Accuracy   '])
all_results[' Precision   '] = pd.to_numeric(all_results[' Precision   '])
all_results[' Recall   '] = pd.to_numeric(all_results[' Recall   '])
all_results[' F1-score   '] = pd.to_numeric(all_results[' F1-score   '])
all_results[' Roc AUC   '] = pd.to_numeric(all_results[' Roc AUC   '])

all_results['   Threshold '] = pd.to_numeric(all_results['   Threshold '])

In [ ]:
all_results[['   Threshold ', ' Accuracy   ', ' Precision   ', ' Recall   ', ' F1-score   ']]

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming df is your DataFrame
# You can load your data using df = pd.read_csv('your_data.csv') or create it programmatically

# Select relevant columns for boxplots
metrics = [ ' Accuracy   ', ' Precision   ', ' Recall   ', ' F1-score   ']
selected_df = all_results[[' Algorithm                                              '] + metrics]

# Melt the DataFrame to long format for seaborn
melted_df = pd.melt(selected_df, id_vars=[' Algorithm                                              '], value_vars=metrics, var_name='Metric', value_name='Value')

# Set up the plot
plt.figure(figsize=(50, 28))
sns.set_theme(style="whitegrid")

# Create boxplot using seaborn
sns.boxplot(x='Metric', y='Value', hue=' Algorithm                                              ', data=melted_df, palette="Set3")

# Add labels and title
plt.xlabel('Metric')
plt.ylabel('Value')
plt.title('Algorithm Performance Comparison')

# Adjust the legend position
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

# Show the plot
plt.show()

##### here is also a good visualization

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming df is your DataFrame
# You can load your data using df = pd.read_csv('your_data.csv') or create it programmatically

# Select relevant columns for boxplots
metrics = [ ' Accuracy   ', ' Precision   ', ' Recall   ', ' F1-score   ']
selected_df = all_results[[' Algorithm                                              '] + metrics]

# Melt the DataFrame to long format for seaborn
melted_df = pd.melt(selected_df, id_vars=[' Algorithm                                              '], value_vars=metrics, var_name='Metric', value_name='Value')

# Set up the plot
plt.figure(figsize=(16, 9))
sns.set_theme(style="whitegrid")

# Create boxplot using seaborn
ax = sns.boxplot(x='Metric', y='Value', hue=' Algorithm                                              ', data=melted_df, palette="Set3")

# Rotate x-axis labels vertically
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')

# Add labels and title
plt.xlabel('Metric')
plt.ylabel('Value')
plt.title('Algorithm Performance Comparison')

# Adjust the legend position
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

# Show the plot
plt.tight_layout()
plt.show()


### that;s a good visualization

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming df is your DataFrame
# You can load your data using df = pd.read_csv('your_data.csv') or create it programmatically

# Select relevant columns for boxplots
metrics = [' Accuracy   ', ' Precision   ', ' Recall   ', ' F1-score   ']
selected_df = all_results[[' Algorithm                                              '] + metrics]

# Melt the DataFrame to long format for seaborn
melted_df = pd.melt(selected_df, id_vars=[' Algorithm                                              '], value_vars=metrics, var_name='Metric', value_name='Value')

# Set up a color palette with unique colors
unique_colors = sns.color_palette('husl', n_colors=len(selected_df[' Algorithm                                              '].unique()))

# Set up the plot
plt.figure(figsize=(16, 9))
sns.set_theme(style="whitegrid")

# Create boxplot using seaborn with unique colors
ax = sns.boxplot(x='Metric', y='Value', hue=' Algorithm                                              ', data=melted_df, palette=unique_colors)

# Rotate x-axis labels vertically
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')

# Add labels and title
plt.xlabel('Metric')
plt.ylabel('Value')
plt.title('Algorithm Performance Comparison')

# Adjust the legend position
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

# Show the plot
plt.tight_layout()
plt.show()

##### below is a good visualization

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming df is your DataFrame
# You can load your data using df = pd.read_csv('your_data.csv') or create it programmatically

# Define the list of algorithms for the two categories
first_category_algorithms = [
    'DecisionTreeRegressor(random_state=42)',
       'RandomForestRegressor(random_state=42)',
    #    'SGDRegressor(random_state=42)',
    #    'KNeighborsRegressor(n_neighbors=3)',
    #    'AdaBoostRegressor(random_state=42)', 'BayesianRidge()',
    #    'ExtraTreesRegressor(random_state=42)',
    #    'BaggingRegressor(random_state=42)',
    #    "LGBMRegressor(objective='regression', random_state=42)",
    #    "SVR(C=0.025, kernel='linear')", 'SVR(C=1, gamma=2)',
    #    'MLPRegressor(random_state=42)',
]

second_category_algorithms = [
    'Survival_Analysis_scores_cph_tree5',
       'Survival_Analysis_scores_cph_tree10',
       'Survival_Analysis_scores_cph_tree25'
]

# Add a new column 'Category' based on algorithm membership
all_results[' Algorithm                                              '] = all_results[' Algorithm                                              '].str.strip()
all_results['Category'] = all_results[' Algorithm                                              '].apply(lambda x: 'Category 1' if x in first_category_algorithms else 'Category 2')

# Select relevant columns for boxplots
metrics = [' Accuracy   ', ' Precision   ', ' Recall   ', ' F1-score   ']
selected_df = all_results[['Category'] + metrics]

# Melt the DataFrame to long format for seaborn
melted_df = pd.melt(selected_df, id_vars=['Category'], value_vars=metrics, var_name='Metric', value_name='Value')

# Set up a color palette with unique colors for each category
palette = {'Category 1': 'lightblue', 'Category 2': 'lightcoral'}

# Set up the plot
plt.figure(figsize=(8, 6))
sns.set_theme(style="whitegrid")

# Create boxplot using seaborn with unique colors
ax = sns.boxplot(x='Metric', y='Value', hue='Category', data=melted_df, palette=palette)

# Rotate x-axis labels vertically
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')

# Add labels and title
plt.xlabel('Metric')
plt.ylabel('Value')
plt.title('Algorithm Performance Comparison (Category 1 vs Category 2)')

# Adjust the legend position
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

# Show the plot
plt.tight_layout()
plt.show()

#### Visualization based on algorithm

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Select relevant columns for boxplots
metrics = ['   accuracy ','   precision ', '    recall ', '        f1 ']
selected_df = new_results[[' Algorithm                            '] + metrics]

# Melt the DataFrame to long format for seaborn
melted_df = pd.melt(selected_df, id_vars=[' Algorithm                            '], value_vars=metrics, var_name='Metric', value_name='Value')

# Set up a color palette with unique colors
unique_colors = sns.color_palette('husl', n_colors=len(selected_df[' Algorithm                            '].unique()))

# Set up the plot
plt.figure(figsize=(50, 30))
sns.set_theme(style="whitegrid")

# Create boxplot using seaborn with unique colors
ax = sns.boxplot(x='Metric', y='Value', hue=' Algorithm                            ', data=melted_df, palette=unique_colors)

# Rotate x-axis labels vertically
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')

# Add labels and title
plt.xlabel('Metric')
plt.ylabel('Value')
plt.title('Algorithm Performance Comparison')

# Adjust the legend position
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os 

# Assuming df is your DataFrame
# You can load your data using df = pd.read_csv('your_data.csv') or create it programmatically


# Define the list of algorithms for the two categories
first_category_algorithms = [
    # 'scores_cph_tree5_risk_prediction',
    #    'scores_cph_tree6_risk_prediction',
    #    'scores_cph_tree7_risk_prediction',
    #    'scores_cph_tree8_risk_prediction',
    #    'scores_cph_tree9_risk_prediction',
    #    'scores_cph_tree10_risk_prediction',
    #    'scores_cph_tree11_risk_prediction',
    #    'scores_cph_tree12_risk_prediction',
    #    'scores_cph_tree13_risk_prediction',
    #    'scores_cph_tree14_risk_prediction',
    #    'scores_cph_tree15_risk_prediction',
    #    'scores_cph_tree16_risk_prediction',
    #    'scores_cph_tree17_risk_prediction',
    #    'scores_cph_tree18_risk_prediction',
    #    'scores_cph_tree19_risk_prediction',
    #    'scores_cph_tree20_risk_prediction',
    #    'scores_cph_tree21_risk_prediction',
    #    'scores_cph_tree22_risk_prediction',
    #    'scores_cph_tree23_risk_prediction',
    #    'scores_cph_tree24_risk_prediction',
    #    'scores_cph_tree25_risk_prediction',
    #    'scores_cph_tree26_risk_prediction',
    #    'scores_cph_tree27_risk_prediction',
    #    'scores_cph_tree28_risk_prediction',
    #    'scores_cph_tree29_risk_prediction',
    #    'scores_cph_tree30_risk_prediction',
    #    'scores_cph_tree31_risk_prediction',
    #    'scores_cph_tree32_risk_prediction',
    #    'scores_cph_tree33_risk_prediction',
    #    'scores_cph_tree34_risk_prediction',
    #    'scores_cph_tree35_risk_prediction',
    #    'scores_cph_tree36_risk_prediction',
    #    'scores_cph_tree37_risk_prediction',
    #    'scores_cph_tree38_risk_prediction',
    #    'scores_cph_tree39_risk_prediction',
    #    'scores_cph_tree40_risk_prediction',
    #    'scores_cph_tree41_risk_prediction',
    #    'scores_cph_tree42_risk_prediction',
    #    'scores_cph_tree43_risk_prediction',
    #    'scores_cph_tree44_risk_prediction',
    #    'scores_cph_tree45_risk_prediction',
    #    'scores_cph_tree46_risk_prediction',
    #    'scores_cph_tree47_risk_prediction',
    #    'scores_cph_tree48_risk_prediction',
    #    'scores_cph_tree49_risk_prediction',
    #    'scores_cph_tree50_risk_prediction',
    #    'scores_cph_tree51_risk_prediction',
    #    'scores_cph_tree52_risk_prediction',
    #    'scores_cph_tree53_risk_prediction',
    #    'scores_cph_tree54_risk_prediction',
    #    'scores_cph_tree55_risk_prediction',
    #    'scores_cph_tree56_risk_prediction',
    #    'scores_cph_tree57_risk_prediction',
    #    'scores_cph_tree58_risk_prediction',
    #    'scores_cph_tree59_risk_prediction',
    #    'scores_cph_tree60_risk_prediction',

       'CoxPHSurvivalAnalysis_risk_prediction'

]

second_category_algorithms = [
    'scores_cph_tree5_survival_prob_adjusted',
       'scores_cph_tree10_mean_survival_prob_adjusted',
       'scores_cph_tree15_mean_survival_prob_adjusted',
       'scores_cph_tree20_mean_survival_prob_adjusted',
       'scores_cph_tree25_mean_survival_prob_adjusted',
       'scores_cph_tree30_mean_survival_prob_adjusted',
       'scores_cph_tree35_mean_survival_prob_adjusted',
       'scores_cph_tree40_mean_survival_prob_adjusted',
       'scores_cph_tree45_mean_survival_prob_adjusted',
       'scores_cph_tree50_mean_survival_prob_adjusted',
       'scores_cph_tree55_mean_survival_prob_adjusted',
       'scores_cph_tree60_mean_survival_prob_adjusted'

]

third_category_algorithms = [
    'LinearRegressionBinaryCLF'
    ]

fourth_category_algorithms = [
    'LinearRegressionBinaryCLF_Cerceira'
    ]

fifth_category_algorithms = [
    # 'scores_cph_tree5_mean_survival_prob',
    #    'scores_cph_tree10_mean_survival_prob',
    #    'scores_cph_tree15_mean_survival_prob',
    #    'scores_cph_tree20_mean_survival_prob',
    #    'scores_cph_tree25_mean_survival_prob',
    #    'scores_cph_tree30_mean_survival_prob',
    #    'scores_cph_tree35_mean_survival_prob',
    #    'scores_cph_tree40_mean_survival_prob',
    #    'scores_cph_tree45_mean_survival_prob',
    #    'scores_cph_tree50_mean_survival_prob',
    #    'scores_cph_tree55_mean_survival_prob',
    #    'scores_cph_tree60_mean_survival_prob'

    ' scores_cph_tree60_survival_prob',
       'scores_cph_tree5_mean_survival_prob',
       'scores_cph_tree10_mean_survival_prob',
       'scores_cph_tree15_mean_survival_prob',
       'scores_cph_tree20_mean_survival_prob',
       'scores_cph_tree25_mean_survival_prob',
       'scores_cph_tree30_mean_survival_prob',
       'scores_cph_tree35_mean_survival_prob',
       'scores_cph_tree40_mean_survival_prob',
       'scores_cph_tree45_mean_survival_prob',
       'scores_cph_tree50_mean_survival_prob',
       'scores_cph_tree55_mean_survival_prob',
       'scores_cph_tree60_mean_survival_prob'

       ]

# Add a new column 'Category' based on algorithm membership
new_results[' Algorithm                                     '] = new_results[' Algorithm                                     '].str.strip()

#new_results['Category'] = new_results[' Algorithm                         '].apply(lambda x: 'Category 1' if x in first_category_algorithms else 'Category 2')

#new_results['Category'] = new_results[' Algorithm                         '].apply(lambda x: 'Risk Prediction' if x in first_category_algorithms else ('Survival Probability' if x in second_category_algorithms else 'Binary Classifier'))

new_results['Category'] = new_results[' Algorithm                                     '].apply(lambda x: 
    'Risk Prediction' if x in first_category_algorithms else 
    ('Mean Survival Probability' if x in fifth_category_algorithms else 
    ('Binary Classifier' if x in third_category_algorithms else 
    'Cerceira Binary Classifier')))

# new_results['Category'] = new_results[' Algorithm                            '].apply(lambda x: 
#     'Risk Prediction' if x in first_category_algorithms else 
#     ('Survival Probability' if x in second_category_algorithms else 
#     ('Binary Classifier' if x in third_category_algorithms else 
#     ('Cerceira Binary Classifier' if x in fourth_category_algorithms else 
#     'Mean Survival Probability'))))


# Select relevant columns for boxplots
metrics = ['   accuracy ','   precision ', '    recall ', '        f1 ']
selected_df = new_results[['Category'] + metrics]

# Melt the DataFrame to long format for seaborn
melted_df = pd.melt(selected_df, id_vars=['Category'], value_vars=metrics, var_name='Metric', value_name='Value')

# Set up a color palette with unique colors for each category
palette = {'Risk Prediction': 'lightgreen', 
           'Survival Probability': 'lightgrey',  
           'Binary Classifier': 'lightblue', 
           'Cerceira Binary Classifier': 'lightyellow', 
           'Mean Survival Probability':'lightcoral'}

# Set up the plot
plt.figure(figsize=(16, 10))
sns.set_theme(style="whitegrid")

# Create boxplot using seaborn with unique colors
ax = sns.boxplot(x='Metric', y='Value', hue='Category', data=melted_df, palette=palette)

# Rotate x-axis labels vertically
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')

# Add labels and title
plt.xlabel('Metric')
plt.ylabel('Value')
plt.title({algorithm_name})

# Adjust the legend position
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

# Show the plot
plt.tight_layout()
file_path = os.path.join('/home/gmarinos/Documents/Code/extended_version_1st_pub/plots', algorithm_name)
plt.savefig(file_path)
plt.show()


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os 


# Assuming df is your DataFrame
# You can load your data using df = pd.read_csv('your_data.csv') or create it programmatically


# Define the list of algorithms for the two categories
first_category_algorithms = [
    # 'scores_cph_tree5_risk_prediction',
    #    'scores_cph_tree6_risk_prediction',
    #    'scores_cph_tree7_risk_prediction',
    #    'scores_cph_tree8_risk_prediction',
    #    'scores_cph_tree9_risk_prediction',
    #    'scores_cph_tree10_risk_prediction',
    #    'scores_cph_tree11_risk_prediction',
    #    'scores_cph_tree12_risk_prediction',
    #    'scores_cph_tree13_risk_prediction',
    #    'scores_cph_tree14_risk_prediction',
    #    'scores_cph_tree15_risk_prediction',
    #    'scores_cph_tree16_risk_prediction',
    #    'scores_cph_tree17_risk_prediction',
    #    'scores_cph_tree18_risk_prediction',
    #    'scores_cph_tree19_risk_prediction',
    #    'scores_cph_tree20_risk_prediction',
    #    'scores_cph_tree21_risk_prediction',
    #    'scores_cph_tree22_risk_prediction',
    #    'scores_cph_tree23_risk_prediction',
    #    'scores_cph_tree24_risk_prediction',
    #    'scores_cph_tree25_risk_prediction',
    #    'scores_cph_tree26_risk_prediction',
    #    'scores_cph_tree27_risk_prediction',
    #    'scores_cph_tree28_risk_prediction',
    #    'scores_cph_tree29_risk_prediction',
    #    'scores_cph_tree30_risk_prediction',
    #    'scores_cph_tree31_risk_prediction',
    #    'scores_cph_tree32_risk_prediction',
    #    'scores_cph_tree33_risk_prediction',
    #    'scores_cph_tree34_risk_prediction',
    #    'scores_cph_tree35_risk_prediction',
    #    'scores_cph_tree36_risk_prediction',
    #    'scores_cph_tree37_risk_prediction',
    #    'scores_cph_tree38_risk_prediction',
    #    'scores_cph_tree39_risk_prediction',
    #    'scores_cph_tree40_risk_prediction',
    #    'scores_cph_tree41_risk_prediction',
    #    'scores_cph_tree42_risk_prediction',
    #    'scores_cph_tree43_risk_prediction',
    #    'scores_cph_tree44_risk_prediction',
    #    'scores_cph_tree45_risk_prediction',
    #    'scores_cph_tree46_risk_prediction',
    #    'scores_cph_tree47_risk_prediction',
    #    'scores_cph_tree48_risk_prediction',
    #    'scores_cph_tree49_risk_prediction',
    #    'scores_cph_tree50_risk_prediction',
    #    'scores_cph_tree51_risk_prediction',
    #    'scores_cph_tree52_risk_prediction',
    #    'scores_cph_tree53_risk_prediction',
    #    'scores_cph_tree54_risk_prediction',
    #    'scores_cph_tree55_risk_prediction',
    #    'scores_cph_tree56_risk_prediction',
    #    'scores_cph_tree57_risk_prediction',
    #    'scores_cph_tree58_risk_prediction',
    #    'scores_cph_tree59_risk_prediction',
    #    'scores_cph_tree60_risk_prediction',

    'scores_cph_tree5_risk_prediction',
       'scores_cph_tree10_risk_prediction',
       'scores_cph_tree15_risk_prediction',
       'scores_cph_tree20_risk_prediction',
       'scores_cph_tree25_risk_prediction',
       'scores_cph_tree30_risk_prediction',
       'scores_cph_tree35_risk_prediction',
       'scores_cph_tree40_risk_prediction',
       'scores_cph_tree45_risk_prediction',
       'scores_cph_tree50_risk_prediction',
       'scores_cph_tree55_risk_prediction',
       'scores_cph_tree60_risk_prediction'
]

second_category_algorithms = [
    'scores_cph_tree5_survival_prob',
       'scores_cph_tree10_survival_prob',
       'scores_cph_tree15_survival_prob',
       'scores_cph_tree20_survival_prob',
       'scores_cph_tree25_survival_prob',
       'scores_cph_tree30_survival_prob',
       'scores_cph_tree35_survival_prob',
       'scores_cph_tree40_survival_prob',
       'scores_cph_tree45_survival_prob',
       'scores_cph_tree50_survival_prob',
       'scores_cph_tree55_survival_prob',
       'scores_cph_tree60_survival_prob'
]

third_category_algorithms = [
    'Gradient_Boosting_CLF'
    ]

fourth_category_algorithms = [
    'Gradient_Boosting_Cerceira'
    ]

fifth_category_algorithms = [
    'scores_cph_tree5_mean_survival_prob',
       'scores_cph_tree10_mean_survival_prob',
       'scores_cph_tree15_mean_survival_prob',
       'scores_cph_tree20_mean_survival_prob',
       'scores_cph_tree25_mean_survival_prob',
       'scores_cph_tree30_mean_survival_prob',
       'scores_cph_tree35_mean_survival_prob',
       'scores_cph_tree40_mean_survival_prob',
       'scores_cph_tree45_mean_survival_prob',
       'scores_cph_tree50_mean_survival_prob',
       'scores_cph_tree55_mean_survival_prob',
       'scores_cph_tree60_mean_survival_prob'
       ]


sixth_category_algorithms = [
    'scores_cph_tree5_mean_survival_prob_adjusted',
       'scores_cph_tree10_mean_survival_prob_adjusted',
       'scores_cph_tree15_mean_survival_prob_adjusted',
       'scores_cph_tree20_mean_survival_prob_adjusted',
       'scores_cph_tree25_mean_survival_prob_adjusted',
       'scores_cph_tree30_mean_survival_prob_adjusted',
       'scores_cph_tree35_mean_survival_prob_adjusted',
       'scores_cph_tree40_mean_survival_prob_adjusted',
       'scores_cph_tree45_mean_survival_prob_adjusted',
       'scores_cph_tree50_mean_survival_prob_adjusted',
       'scores_cph_tree55_mean_survival_prob_adjusted',
       'scores_cph_tree60_mean_survival_prob_adjusted'
       ]



# Add a new column 'Category' based on algorithm membership
new_results[' Algorithm                                     '] = new_results[' Algorithm                                     '].str.strip()

#new_results['Category'] = new_results[' Algorithm                         '].apply(lambda x: 'Category 1' if x in first_category_algorithms else 'Category 2')

#new_results['Category'] = new_results[' Algorithm                         '].apply(lambda x: 'Risk Prediction' if x in first_category_algorithms else ('Survival Probability' if x in second_category_algorithms else 'Binary Classifier'))

# new_results['Category'] = new_results[' Algorithm                                     '].apply(lambda x: 
#     'Risk Prediction' if x in first_category_algorithms else 
#     ('Mean Survival Probability' if x in fifth_category_algorithms else 
#     ('Binary Classifier' if x in third_category_algorithms else 
#     'Cerceira Binary Classifier')))

new_results['Category'] = new_results[' Algorithm                                     '].apply(lambda x: 
    'Risk Prediction' if x in first_category_algorithms else 
    ('Old Survival Probability' if x in second_category_algorithms else 
    ('Binary Classifier' if x in third_category_algorithms else 
    ('Cerceira Binary Classifier' if x in fourth_category_algorithms else 
    ('Mean Survival Probability' if x in fifth_category_algorithms else 
    'Mean Survival Probability Adjusted')))))



# Select relevant columns for boxplots
metrics = ['   accuracy ','   precision ', '    recall ', '        f1 ']
selected_df = new_results[['Category'] + metrics]

# Melt the DataFrame to long format for seaborn
melted_df = pd.melt(selected_df, id_vars=['Category'], value_vars=metrics, var_name='Metric', value_name='Value')

# Set up a color palette with unique colors for each category
palette = {'Risk Prediction': 'lightgreen', 
           'Old Survival Probability': 'lightgrey',  
           'Binary Classifier': 'lightblue', 
           'Cerceira Binary Classifier': 'lightyellow', 
           'Mean Survival Probability':'lightcoral', 
           'Mean Survival Probability Adjusted':'black'}

# Set up the plot
plt.figure(figsize=(16, 10))
sns.set_theme(style="whitegrid")

# Create boxplot using seaborn with unique colors
ax = sns.boxplot(x='Metric', y='Value', hue='Category', data=melted_df, palette=palette)

# Rotate x-axis labels vertically
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')

# Add labels and title
plt.xlabel('Metric')
plt.ylabel('Value')
plt.title(algorithm_name)

# Adjust the legend position
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

# Show the plot
plt.tight_layout()
file_path = os.path.join('/home/gmarinos/Documents/Code/extended_version_1st_pub/plots', algorithm_name)
plt.savefig(file_path)
plt.show()

##### only two categories plot 

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming df is your DataFrame
# You can load your data using df = pd.read_csv('your_data.csv') or create it programmatically

# Define the list of algorithms for the two categories
first_category_algorithms = [
    'DecisionTreeRegressor(random_state=42)',
    'RandomForestRegressor(random_state=42)',
    #    'SGDRegressor(random_state=42)',
    #    'KNeighborsRegressor(n_neighbors=3)',
    #    'AdaBoostRegressor(random_state=42)', 'BayesianRidge()',
    #    'ExtraTreesRegressor(random_state=42)',
    #    'BaggingRegressor(random_state=42)',
    #    "LGBMRegressor(objective='regression', random_state=42)",
    #    "SVR(C=0.025, kernel='linear')", 'SVR(C=1, gamma=2)',
    #    'MLPRegressor(random_state=42)',
]

second_category_algorithms = [
    'Survival_Analysis_scores_cph_tree5',
       'Survival_Analysis_scores_cph_tree10',
       'Survival_Analysis_scores_cph_tree25'
]

# Add a new column 'Category' based on algorithm membership
all_results[' Algorithm                                              '] = all_results[' Algorithm                                              '].str.strip()
all_results['Category'] = all_results[' Algorithm                                              '].apply(lambda x: 'Category 1' if x in first_category_algorithms else 'Category 2')

# Select relevant columns for boxplots
metrics = [' Accuracy   ', ' Precision   ', ' Recall   ', ' F1-score   ']
selected_df = all_results[['Category'] + metrics]

# Melt the DataFrame to long format for seaborn
melted_df = pd.melt(selected_df, id_vars=['Category'], value_vars=metrics, var_name='Metric', value_name='Value')

# Set up a color palette with unique colors for each category
palette = {'Category 1': 'lightblue', 'Category 2': 'lightcoral'}

# Set up the plot
plt.figure(figsize=(8, 6))
sns.set_theme(style="whitegrid")

# Create boxplot using seaborn with unique colors
ax = sns.boxplot(x='Metric', y='Value', hue='Category', data=melted_df, palette=palette)

# Rotate x-axis labels vertically
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')

# Add labels and title
plt.xlabel('Metric')
plt.ylabel('Value')
plt.title('Algorithm Performance Comparison (Category 1 vs Category 2)')

# Adjust the legend position
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

# Show the plot
plt.tight_layout()
plt.show()

#### In case of a simple/brief experiment use that 

In [ ]:
from sksurv.ensemble import RandomSurvivalForest, ExtraSurvivalTrees
from sksurv.tree import SurvivalTree
from sksurv.svm import FastKernelSurvivalSVM, FastSurvivalSVM, NaiveSurvivalSVM
from sksurv.linear_model import CoxPHSurvivalAnalysis
import pickle 
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score, roc_auc_score
from scipy.stats import norm, laplace, logistic, gumbel_r, lognorm, cauchy, genextreme 
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.svm import SVC, SVR
from sklearn.linear_model import LinearRegression, LogisticRegression

random_state = 42

scores_RandomSurvivalForest = {}

results = []

distributions = [genextreme, norm, laplace, logistic, gumbel_r, lognorm, cauchy]

predictions = {}

# Loop through models and distributions and evaluate each one
for name, values in splitted_data.items():

    if name == 'fruits and vegetables':
    
        X_train_survival = values['X_train_survival']
        X_test_survival = values['X_test_survival']
        y_train_survival = values['y_train_survival']
        y_test_survival = values['y_test_survival']

        X_train = values['X_train']
        y_train = values['y_train']
        X_test = values['X_test']
        y_test = values['y_test']

        X_train_mn_ext_feat = values['X_train_mn_ext_feat']
        X_test_mn_ext_feat = values['X_test_mn_ext_feat']
        y_train_mn_ext_feat = values['y_train_mn_ext_feat'] 
        y_test_mn_ext_feat = values['y_test_mn_ext_feat']
        
        X_train_tsfel = values['X_train_tsfel']
        y_train_tsfel = values['y_train_tsfel']
        X_test_tsfel = values['X_test_tsfel']
        y_test_tsfel = values['y_test_tsfel']

        THRESHOLD = round(dataframes[name]['threshold'].iloc[0])


        ## Risk Prediction approach 
        est_cph_tree = RandomSurvivalForest()
        
        est_cph_tree.fit(X_train_survival, y_train_survival)
        # print('c-index:', est_cph_tree.score(X_test_survival, y_test_survival))


        metrics = []

        y_test_ = (y_test_survival['extreme_event'] == True).astype(int)

        for i in range(5, 61, 5):
            list_of_lists = est_cph_tree.predict_survival_function(X_test_survival, return_array=True)

            y_test_ = (y_test_survival['extreme_event'] == True).astype(int)

            #Approach 3
            predicted_mean_survival_prob = []

            for prob in list_of_lists:
                predicted_mean_survival_prob.append(np.mean(prob))

            
            y_pred_binary = (np.array(predicted_mean_survival_prob) > 0.4).astype(int)


            accuracy = accuracy_score(y_test_, y_pred_binary)
            precision = precision_score(y_test_, y_pred_binary)
            recall = recall_score(y_test_, y_pred_binary)
            f1 = f1_score(y_test_, y_pred_binary)
            roc_auc = roc_auc_score(y_test_, list_of_lists[0:,0])

            print('Model:', 'scores_cph_tree'+str(i)+'_survival_prob',
                'c-index:', np.round(est_cph_tree.score(X_test_survival, y_test_survival),2),
                'accuracy:', accuracy, 
                'precision:', precision, 
                'recall:', recall, 
                'f1-score:', f1, 
                'roc_auc:', roc_auc)